# Install Package

In [ ]:
!pip install transformers
!pip install tensorflow_hub

# Import Module

In [7]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
import transformers
import tensorflow_hub as hub
from transformers import GPT2Tokenizer, TFGPT2Model, AutoTokenizer, BertTokenizer, TFBertModel
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Input, Dropout, Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [3]:
df = pd.read_csv('Petition_data_advised.csv')
df.head()

,content,label
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1


# Train, Test Split

In [4]:
train_data = pd.concat([df[df['label'] == 1][:80], df[df['label'] == 0][:400]])
test_data = pd.concat([df[df['label'] == 1][80:], df[df['label'] == 0][400:]])

# Data Preprocessing

In [5]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [8]:
max_seq_len = 128
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

X_train, y_train = convert_examples_to_features(train_data['content'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data['content'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 59/59 [00:00<00:00, 229.55it/s]


# Modeling

In [9]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [10]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [11]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [12]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base")
  
  earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

  checkpoint = ModelCheckpoint('BERT_kor.h5',
                              save_best_only=True,
                              save_weights_only=True,
                              monitor='val_loss',
                              mode='min',
                              verbose=False)

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [13]:
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
15/15 [==============================] - 77s 902ms/step - loss: 0.8034 - accuracy: 0.7917 - val_loss: 0.7344 - val_accuracy: 0.7119
Epoch 2/50
15/15 [==============================] - 3s 239ms/step - loss: 0.4871 - accuracy: 0.8333 - val_loss: 0.6762 - val_accuracy: 0.7119
Epoch 3/50
15/15 [==============================] - 2s 118ms/step - loss: 0.4628 - accuracy: 0.8333 - val_loss: 0.7103 - val_accuracy: 0.7119
Epoch 4/50
15/15 [==============================] - 3s 240ms/step - loss: 0.4597 - accuracy: 0.8333 - val_loss: 0.6511 - val_accuracy: 0.7119
Epoch 5/50
15/15 [==============================] - 4s 244ms/step - loss: 0.4513 - accuracy: 0.8333 - val_loss: 0.6296 - val_accuracy: 0.7119
Epoch 6/50
15/15 [==============================] - 4s 244ms/step - loss: 0.4564 - accuracy: 0.8333 - val_loss: 0.6018 - val_accuracy: 0.7119
Epoch 7/50
15/15 [==============================] - 2s 116ms/step - loss: 0.4892 - accuracy: 0.8333 - val_loss: 0.7654 - val_accuracy: 0.7119
Epoch

In [14]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('BERT_kor.h5')
model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 34ms/step - loss: 0.6018 - accuracy: 0.7119


[0.6018162369728088, 0.7118644118309021]

In [ ]:
# 예측
model.predict(X_test)[0]

2/2 [==============================] - 7s 3s/step


array([0.3450919], dtype=float32)